In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score as R2
from sklearn.metrics import mean_squared_error as MSE

In [2]:
rest = pd.read_csv('/Volumes/Database/Research/ozone-budget/result_df_UKESM1_3_0_NN_width_64_dropout_0.1_90.csv')
data = pd.read_csv('/Volumes/Database/Research/C7 Multivariate analysis/UKESM1-0-LL_diag_2014(1217).csv')

In [94]:
rest['tas'] = data['tas']
rest['pan'] = data['pan']
rest['photo1d'] = data['photo1d']

rest['o3_mole'] = rest['o3']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['no_mole'] = rest['no']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['no2_mole'] = rest['no2']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['oh_mole'] = rest['oh']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['h2o_mole'] = rest['h2o']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['hno3_mole'] = rest['hno3']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['pan_mole'] = rest['pan']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)

rest['ho2_mole'] = rest['HO₂']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)/3.33
rest['ch3o2_mole'] = rest['CH₃O₂']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)/50
rest['o1d_mole'] = rest['O¹D']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)

rest['o3prod_mole'] = rest['o3prod']*(6.022*10**23)/(10**6)
rest['o3loss_mole'] = rest['o3loss']*(6.022*10**23)/(10**6)

rest['k_ho2_ho2'] = 2.2*10**(-13)*np.exp(600/rest['tas'])
rest['k_oh_no2'] = 6.5*10**(-11)
rest['k_no_ho2'] = 3.30*10**(-12)*np.exp(270/rest['tas'])
rest['k_no_ch3o2'] = 2.30*10**(-12)*np.exp(360/rest['tas'])
rest['k_o1d_h2o'] = 1.63*10**(-10)*np.exp(60/rest['tas'])
rest['k_o3_ho2'] = 2.03*10**(-16)*(rest['tas']/300)**4.57*np.exp(693/rest['tas'])
rest['k_o3_oh'] = data['k_o3_oh']

### First, it is better to compare the kinetic parameters used in UKESM1-0-LL with the IUPAC preference. It is expected that the temperature-dependent kinetic rates should be close to the IUPAC suggested values. 

In [95]:
rest['k_o3_oh'].describe()

count    9.720000e+05
mean     5.872950e-14
std      1.461017e-14
min      1.399188e-14
25%      5.085895e-14
50%      6.167296e-14
75%      7.141854e-14
max      8.512149e-14
Name: k_o3_oh, dtype: float64

In [96]:
rest['k_o3_ho2'].describe()

count    9.720000e+05
mean     1.759353e-15
std      2.646843e-16
min      9.950170e-16
25%      1.607599e-15
50%      1.805485e-15
75%      1.992593e-15
max      2.271330e-15
Name: k_o3_ho2, dtype: float64

In [97]:
rest['k_o1d_h2o'].describe()

count    9.720000e+05
mean     2.026049e-10
std      3.926133e-12
min      1.973297e-10
25%      1.995529e-10
50%      2.014304e-10
75%      2.039245e-10
max      2.214346e-10
Name: k_o1d_h2o, dtype: float64

In [98]:
rest['k_no_ho2'].describe()

count    9.720000e+05
mean     8.808589e-12
std      8.049641e-13
min      7.798945e-12
25%      8.202217e-12
50%      8.555249e-12
75%      9.042365e-12
max      1.310009e-11
Name: k_no_ho2, dtype: float64

In [99]:
rest['k_no_ch3o2'].describe()

count    9.720000e+05
mean     8.531658e-12
std      1.061141e-12
min      7.240309e-12
25%      7.743743e-12
50%      8.191300e-12
75%      8.818986e-12
max      1.445689e-11
Name: k_no_ch3o2, dtype: float64

In [100]:
rest['Pt_O3_OH'] = rest['k_o3_oh']*rest['o3_mole']*rest['oh_mole']
rest['Pr_O3_OH'] = rest['Pt_O3_OH']/rest['o3loss_mole']
rest['Pr_O3_OH'].mean()

0.10613192063156386

### Part 1: HO${_2}$ + NO → NO${_2}$ + OH

In [101]:
rest['ho2_mole'].mean()/rest['oh_mole'].mean()

101.31289217465275

In [102]:
rest['Pt_HO2_NO'] = rest['k_no_ho2']*rest['ho2_mole']*rest['no_mole']
rest['Pr_HO2_NO'] = rest['Pt_HO2_NO']/rest['o3prod_mole']
rest['Pr_HO2_NO'].mean()

0.6773993143029338

### Part 2: CH${_3}$O${_2}$ + NO → NO${_2}$ + HCHO

In [103]:
rest['ho2_mole'].mean()/rest['ch3o2_mole'].mean()

2.9604062734493155

In [104]:
rest['Pt_CH3O2_NO'] = rest['k_no_ch3o2']*rest['ch3o2_mole']*rest['no_mole']
rest['Pr_CH3O2_NO'] = rest['Pt_CH3O2_NO']/rest['o3prod_mole']
rest['Pr_CH3O2_NO'].mean()

0.21756531117856875

### Part 3: RO${_2}$ + NO → NO${_2}$ + RO

In [ ]:
rest['Pt_RO2_NO']